## Introduction to Mesh Parameterization: Exercises
In this notebook you will perform the same analysis you did in 101 on more complex meshes, and try your hand at more complicated parameterization techniques. 

### Part 1: Fixed Boundary Parameterization and Distortion Analysis ###
In this folder you are given 2 meshes -- halfbunny.obj and ogre.obj. Load each of these meshes using gpytoolbox and use the code from notebook 101 to perform the analyses in the next few code blocks. 

In [40]:
# Loading required packages
import numpy as np
import polyscope as ps

#### 1.1 halfbunny.obj

In [41]:
import gpytoolbox as gpy

### TODO: use gpytoolbox to load in halfbunny.obj
MESH_VERTICES, MESH_FACES = gpy.read_mesh("halfbunny.obj")

### TODO: Use the below code copied from notebook 101 to get the boundary and non-boundary edges of the imported mesh
from igl import boundary_loop

bnd = boundary_loop(MESH_FACES)
boundary_idxs = list(sorted(bnd))

# NOTE: pin the boundary to a circle -- can no longer define this by hand
from igl import map_vertices_to_circle
boundary_positions = map_vertices_to_circle(MESH_VERTICES, bnd).astype(MESH_VERTICES.dtype)

# NOTE: Resort the positions to match the order of boundary_idxs
boundary_positions = boundary_positions[np.argsort(bnd)]

# vertices that aren't pinned
pred_idxs = np.array([i for i in range(MESH_VERTICES.shape[0]) if i not in boundary_idxs])

# # Get edge array (unique edges + counts)
from collections import defaultdict
edges = defaultdict(int)
for f in MESH_FACES:
    for i in range(3):
        if f[i] > f[(i+1)%3]:
            edges[(f[(i+1)%3], f[i])] += 1
        else:
            edges[(f[i], f[(i+1)%3])] += 1

# # Valid edges are the ones that are shared by two faces
tot_edges = np.array(list(edges.keys()))
valid_edges = np.array([k for k, v in edges.items() if v == 2])
boundary_edges = np.array([k for k, v in edges.items() if v == 1])

Note in the example code that the process for computing the fixed boundary parameterizations for these meshes will be almost exactly the same as in exercise 101, except it is no longer so trivial to define the boundary positions. Instead, we will make use of libigl to fix the boundary vertices to a circle (standard convex domain). This is done using these additional lines in the example code

```
from igl import map_vertices_to_circle
boundary_positions = map_vertices_to_circle(MESH_VERTICES, bnd).astype(MESH_VERTICES.dtype)
boundary_positions = boundary_positions[np.argsort(bnd)]
```

In [42]:
# TODO: Copy over the setup_parameterization_matrices() function from notebook 101 and use it to compute
# 1) The Tutte parameterization (all weights of 1)
# 2) The Mean Value weights parameterization (see formula and code for computing the mean value weights from notebook 101)

def setup_parameterization_matrices(vertices, boundary_idxs, edges, weights=1):
    """ Set up the Tutte linear system (laplacian weights of 1)

    vertices (np.ndarray): V x 3 array of vertex positions
    boundary_idxs (np.ndarray): B array of boundary vertex indices
    edges (np.ndarray): E x 2 array of edge indices for weight assignment
    weights (float or np.ndarray): E array of edge weights or scalar value (default 1)

    Returns:
        L (np.ndarray): V x V Laplacian matrix with boundary values set to 0
        Lb (np.ndarray): V x B boundary Laplacian
    """

    # Initialize the laplacian matrix
    L = np.zeros((vertices.shape[0], vertices.shape[0]))

    L[edges[:, 0], edges[:, 1]] = weights
    L[edges[:, 1], edges[:, 0]] = weights

    # Off-diagonal elements are negative and diagonal is the negative sum of the row
    L = np.diag(np.sum(L, axis=1)) - L

    # Boundary weights are positive
    Lb = -L[:, boundary_idxs]

    # Boundary diagonal should be set to 0
    Lb[boundary_idxs, range(len(boundary_idxs))] = 0

    # Set the off-diagonal boundary columns to 0
    Ldiag = np.diag(L).copy()

    L[:, boundary_idxs] = 0
    np.fill_diagonal(L, Ldiag)

    return L, Lb

L_tutte, Lb_tutte = setup_parameterization_matrices(MESH_VERTICES, boundary_idxs, valid_edges)
uv_tutte = np.linalg.solve(L_tutte, Lb_tutte @ boundary_positions)
uv_tutte[boundary_idxs] = boundary_positions

meanvalues = {}
for fi in range(len(MESH_FACES)):
    f = MESH_FACES[fi]
    for i in range(3):
        v1 = f[i]
        v2 = f[(i+1)%3]
        v3 = f[(i+2)%3]
        e1 = MESH_VERTICES[v2] - MESH_VERTICES[v1]
        e2 = MESH_VERTICES[v3] - MESH_VERTICES[v1]

        b1 = MESH_VERTICES[v1] - MESH_VERTICES[v2]
        b2 = MESH_VERTICES[v3] - MESH_VERTICES[v2]

        assert (v1, v2) not in meanvalues

        alpha = np.arccos(np.dot(e1, e2) / (np.linalg.norm(e1) * np.linalg.norm(e2)))
        beta = np.arccos(np.dot(b1, b2) / (np.linalg.norm(b1) * np.linalg.norm(b2)))

        # Compute (tan(alpha_ij/2), tan(beta_ij/2), r_ij)
        meanvalues[(v1, v2)] = (np.tan(alpha/2),
                            np.tan(beta/2),
                            np.linalg.norm(MESH_VERTICES[v2] - MESH_VERTICES[v1]))

# For each edge pair, compute the mean value weights
mean_value_weights = np.zeros((len(valid_edges),))
for i, edge in enumerate(valid_edges):
    v1, v2 = edge

    assert (v1, v2) in meanvalues and (v2, v1) in meanvalues

    a1, b2, r1 = meanvalues[(v1, v2)]
    a2, b1, r2 = meanvalues[(v2, v1)]

    assert r1 == r2

    # tan(alpha_ij/2) + tan(beta_ji/2)
    mean_value_weights[i] = (a1 + b1)/r1

L_meanv, Lb_meanv = setup_parameterization_matrices(MESH_VERTICES, boundary_idxs, valid_edges, mean_value_weights)
uv_meanv = np.linalg.solve(L_tutte, Lb_tutte @ boundary_positions)
uv_meanv[boundary_idxs] = boundary_positions

In [43]:
import polyscope as ps

face_cols = np.arange(MESH_FACES.shape[0])

# TODO: Visualize the mesh and the computed UV maps using polyscope
ps.init()
bunny = ps.register_surface_mesh("bunny", MESH_VERTICES, MESH_FACES)
tutte = ps.register_surface_mesh("bunny tutte", uv_tutte + np.array([[3, 0]]), MESH_FACES)
meanv = ps.register_surface_mesh("bunny meanv", uv_meanv - np.array([[3, 0]]), MESH_FACES)

bunny.add_scalar_quantity("color", face_cols, "faces", enabled=True)
tutte.add_scalar_quantity("color", face_cols, "faces", enabled=True)
meanv.add_scalar_quantity("color", face_cols, "faces", enabled=True)

ps.show()

In [ ]:
# TODO: Copy over the get_jacobian() function from notebook 101 and use it to
# compute the area, conformal, and isometric distortion for each parameterization.
def get_jacobian(vs, fs, uvmap):
	""" Get jacobian of mesh given an input UV map

	Args:
		vs (np.ndarray): V x 3 array of vertex positions
		fs (np.ndarray): F x 3 integer array of face indices
		uvmap (np.ndarray): V x 2 array of UV coordinates

	Returns:
		_type_: _description_
	"""
	# Visualize distortion
	from igl import grad
	G = np.array(grad(vs, fs).todense())

	# NOTE: currently gradient is organized as X1, X2, X3, ... Y1, Y2, Y3, ... Z1, Z2, Z3 ... reshape to X1, Y1, Z1, ...
	splitind = G.shape[0]//3
	newG = np.zeros_like(G) # F*3 x V
	newG[::3] = G[:splitind]
	newG[1::3] = G[splitind:2*splitind]
	newG[2::3] = G[2*splitind:]

	J = (newG @ uvmap).reshape(-1, 3, 2) # F x 3 x 2
	return J

def compute_energies(J):
	""" returns area, conformal, and isometric energies """

	_, S, _ = np.linalg.svd(J)

	E_area = S[:, 0] * S[:, 1] + 1/(S[:, 0] * S[:, 1]) - 2
	E_conformal = (S[:, 0] - S[:, 1])**2
	E_isometric = S[:, 0]**2 + S[:, 1]**2 + 1/(S[:, 0]**2) + 1/(S[:, 1]**2) - 4

	return E_area, E_conformal, E_isometric

J_tutte = get_jacobian(MESH_VERTICES, MESH_FACES, uv_tutte)
J_meanv = get_jacobian(MESH_VERTICES, MESH_FACES, uv_meanv)

J_sparse = get_jacobian(MESH_VERTICES, MESH_FACES, uv_tutte)

E_tutte = compute_energies(J_tutte)
E_meanv = compute_energies(J_meanv)

1724
True


In [22]:
# TODO: Visualize the computed distortion energies using polyscope
import polyscope as ps

face_cols = np.arange(MESH_FACES.shape[0])

# TODO: Visualize the mesh and the computed UV maps using polyscope
ps.init()
bunny = ps.register_surface_mesh("bunny", MESH_VERTICES, MESH_FACES)
tutte = ps.register_surface_mesh("bunny tutte", uv_tutte + np.array([[3, 0]]), MESH_FACES)
meanv = ps.register_surface_mesh("bunny meanv", uv_meanv - np.array([[3, 0]]), MESH_FACES)

bunny.add_scalar_quantity("color", face_cols, "faces", enabled=False)
tutte.add_scalar_quantity("color", face_cols, "faces", enabled=False)
meanv.add_scalar_quantity("color", face_cols, "faces", enabled=False)

tutte.add_scalar_quantity("area", E_tutte[0], "faces")
tutte.add_scalar_quantity("conformal", E_tutte[1], "faces")
tutte.add_scalar_quantity("isometric", E_tutte[2], "faces")

meanv.add_scalar_quantity("area", E_meanv[0], "faces")
meanv.add_scalar_quantity("conformal", E_meanv[1], "faces")
meanv.add_scalar_quantity("isometric", E_meanv[2], "faces")

ps.show()

#### 1.2 ogre.obj
Note that this is a significantly larger mesh so expect `np.linalg.solve()` to take a few minutes to run! 

In [23]:
import gpytoolbox as gpy

### TODO: use gpytoolbox to load in halfbunny.obj
MESH_VERTICES, MESH_FACES = gpy.read_mesh("ogre.obj")

### TODO: Use the below code copied from notebook 101 to get the boundary and non-boundary edges of the imported mesh
from igl import boundary_loop

bnd = boundary_loop(MESH_FACES)
boundary_idxs = list(sorted(bnd))

# NOTE: pin the boundary to a circle -- can no longer define this by hand
from igl import map_vertices_to_circle
boundary_positions = map_vertices_to_circle(MESH_VERTICES, bnd).astype(MESH_VERTICES.dtype)

# NOTE: Resort the positions to match the order of boundary_idxs
boundary_positions = boundary_positions[np.argsort(bnd)]

# vertices that aren't pinned
pred_idxs = np.array([i for i in range(MESH_VERTICES.shape[0]) if i not in boundary_idxs])

# # Get edge array (unique edges + counts)
from collections import defaultdict
edges = defaultdict(int)
for f in MESH_FACES:
    for i in range(3):
        if f[i] > f[(i+1)%3]:
            edges[(f[(i+1)%3], f[i])] += 1
        else:
            edges[(f[i], f[(i+1)%3])] += 1

# # Valid edges are the ones that are shared by two faces
tot_edges = np.array(list(edges.keys()))
valid_edges = np.array([k for k, v in edges.items() if v == 2])
boundary_edges = np.array([k for k, v in edges.items() if v == 1])

In [24]:
# TODO: Copy over the setup_parameterization_matrices() function from notebook 101 and use it to compute
# 1) The Tutte parameterization (all weights of 1)
# 2) The Mean Value weights parameterization (see formula and code for computing the mean value weights from notebook 101)

def setup_parameterization_matrices(vertices, boundary_idxs, edges, weights=1):
    """ Set up the Tutte linear system (laplacian weights of 1)

    vertices (np.ndarray): V x 3 array of vertex positions
    boundary_idxs (np.ndarray): B array of boundary vertex indices
    edges (np.ndarray): E x 2 array of edge indices for weight assignment
    weights (float or np.ndarray): E array of edge weights or scalar value (default 1)

    Returns:
        L (np.ndarray): V x V Laplacian matrix with boundary values set to 0
        Lb (np.ndarray): V x B boundary Laplacian
    """

    # Initialize the laplacian matrix
    L = np.zeros((vertices.shape[0], vertices.shape[0]))

    L[edges[:, 0], edges[:, 1]] = weights
    L[edges[:, 1], edges[:, 0]] = weights

    # Off-diagonal elements are negative and diagonal is the negative sum of the row
    L = np.diag(np.sum(L, axis=1)) - L

    # Boundary weights are positive
    Lb = -L[:, boundary_idxs]

    # Boundary diagonal should be set to 0
    Lb[boundary_idxs, range(len(boundary_idxs))] = 0

    # Set the off-diagonal boundary columns to 0
    Ldiag = np.diag(L).copy()

    L[:, boundary_idxs] = 0
    np.fill_diagonal(L, Ldiag)

    return L, Lb

L_tutte, Lb_tutte = setup_parameterization_matrices(MESH_VERTICES, boundary_idxs, valid_edges)
uv_tutte = np.linalg.solve(L_tutte, Lb_tutte @ boundary_positions)
uv_tutte[boundary_idxs] = boundary_positions

meanvalues = {}
for fi in range(len(MESH_FACES)):
    f = MESH_FACES[fi]
    for i in range(3):
        v1 = f[i]
        v2 = f[(i+1)%3]
        v3 = f[(i+2)%3]
        e1 = MESH_VERTICES[v2] - MESH_VERTICES[v1]
        e2 = MESH_VERTICES[v3] - MESH_VERTICES[v1]

        b1 = MESH_VERTICES[v1] - MESH_VERTICES[v2]
        b2 = MESH_VERTICES[v3] - MESH_VERTICES[v2]

        assert (v1, v2) not in meanvalues

        alpha = np.arccos(np.dot(e1, e2) / (np.linalg.norm(e1) * np.linalg.norm(e2)))
        beta = np.arccos(np.dot(b1, b2) / (np.linalg.norm(b1) * np.linalg.norm(b2)))

        # Compute (tan(alpha_ij/2), tan(beta_ij/2), r_ij)
        meanvalues[(v1, v2)] = (np.tan(alpha/2),
                            np.tan(beta/2),
                            np.linalg.norm(MESH_VERTICES[v2] - MESH_VERTICES[v1]))

# For each edge pair, compute the mean value weights
mean_value_weights = np.zeros((len(valid_edges),))
for i, edge in enumerate(valid_edges):
    v1, v2 = edge

    assert (v1, v2) in meanvalues and (v2, v1) in meanvalues

    a1, b2, r1 = meanvalues[(v1, v2)]
    a2, b1, r2 = meanvalues[(v2, v1)]

    assert r1 == r2

    # tan(alpha_ij/2) + tan(beta_ji/2)
    mean_value_weights[i] = (a1 + b1)/r1

L_meanv, Lb_meanv = setup_parameterization_matrices(MESH_VERTICES, boundary_idxs, valid_edges, mean_value_weights)
uv_meanv = np.linalg.solve(L_tutte, Lb_tutte @ boundary_positions)
uv_meanv[boundary_idxs] = boundary_positions

In [25]:
import polyscope as ps

face_cols = np.arange(MESH_FACES.shape[0])

# TODO: Visualize the mesh and the computed UV maps using polyscope
ps.init()
bunny = ps.register_surface_mesh("bunny", MESH_VERTICES, MESH_FACES)
tutte = ps.register_surface_mesh("bunny tutte", uv_tutte + np.array([[3, 0]]), MESH_FACES)
meanv = ps.register_surface_mesh("bunny meanv", uv_meanv - np.array([[3, 0]]), MESH_FACES)

bunny.add_scalar_quantity("color", face_cols, "faces", enabled=True)
tutte.add_scalar_quantity("color", face_cols, "faces", enabled=True)
meanv.add_scalar_quantity("color", face_cols, "faces", enabled=True)

ps.show()

In [ ]:
# TODO: Copy over the get_jacobian() function from notebook 101 and use it to
# compute the area, conformal, and isometric distortion for each parameterization.
def get_jacobian_sp(vs, fs, uvmap):
	""" Get jacobian of mesh given an input UV map

	Args:
		vs (np.ndarray): V x 3 array of vertex positions
		fs (np.ndarray): F x 3 integer array of face indices
		uvmap (np.ndarray): V x 2 array of UV coordinates

	Returns:
		_type_: _description_
	"""
	# Visualize distortion
	from igl import grad

	print(fs.shape[0])
	G = grad(vs, fs)

	splitind = G.shape[0]//3

	# NOTE: currently gradient is organized as X1, X2, X3, ... Y1, Y2, Y3, ... Z1, Z2, Z3 ... reshape to X1, Y1, Z1, ...
	# newG = np.zeros_like(G) # F*3 x V
	# newG[::3] = G[:splitind]
	# newG[1::3] = G[splitind:2*splitind]
	# newG[2::3] = G[2*splitind:]

	J = (G * uvmap) # 3F x 2, (dX1, dX2, ...)
	newJ = np.zeros((splitind, 3, 2))
	newJ[:, 0] = J[:splitind]
	newJ[:, 1] = J[splitind:2*splitind]
	newJ[:, 2] = J[2*splitind:]

	return newJ

def compute_energies(J):
	""" returns area, conformal, and isometric energies """

	_, S, _ = np.linalg.svd(J)

	E_area = S[:, 0] * S[:, 1] + 1/(S[:, 0] * S[:, 1]) - 2
	E_conformal = (S[:, 0] - S[:, 1])**2
	E_isometric = S[:, 0]**2 + S[:, 1]**2 + 1/(S[:, 0]**2) + 1/(S[:, 1]**2) - 4

	return E_area, E_conformal, E_isometric

J_tutte = get_jacobian_sp(MESH_VERTICES, MESH_FACES, uv_tutte)
J_meanv = get_jacobian_sp(MESH_VERTICES, MESH_FACES, uv_meanv)

E_tutte = compute_energies(J_tutte)
E_meanv = compute_energies(J_meanv)

39856
39856


In [38]:
# TODO: Visualize the computed distortion energies using polyscope
import polyscope as ps

face_cols = np.arange(MESH_FACES.shape[0])

# TODO: Visualize the mesh and the computed UV maps using polyscope
ps.init()
bunny = ps.register_surface_mesh("bunny", MESH_VERTICES, MESH_FACES)
tutte = ps.register_surface_mesh("bunny tutte", uv_tutte + np.array([[3, 0]]), MESH_FACES)
meanv = ps.register_surface_mesh("bunny meanv", uv_meanv - np.array([[3, 0]]), MESH_FACES)

bunny.add_scalar_quantity("color", face_cols, "faces", enabled=False)
tutte.add_scalar_quantity("color", face_cols, "faces", enabled=False)
meanv.add_scalar_quantity("color", face_cols, "faces", enabled=False)

tutte.add_scalar_quantity("area", E_tutte[0], "faces")
tutte.add_scalar_quantity("conformal", E_tutte[1], "faces")
tutte.add_scalar_quantity("isometric", E_tutte[2], "faces")

meanv.add_scalar_quantity("area", E_meanv[0], "faces")
meanv.add_scalar_quantity("conformal", E_meanv[1], "faces")
meanv.add_scalar_quantity("isometric", E_meanv[2], "faces")

ps.show()

### Part 2: LSCM and ARAP ###
In this part you will use two more advanced parameterization techniques to flatten the same meshes. 

These two methods are Least Squares Conformal Maps [(LSCM)](https://www.cs.jhu.edu/~misha/Fall09/Levy02.pdf) and As-Rigid-As-Possible Mesh Parameterization [(ARAP)](https://cs.harvard.edu/~sjg/papers/arap.pdf). 

As you will see, these are two examples of **free boundary** methods (though LSCM technically requires two vertices to be pinned), which allows the boundary to move independently to perform the desired distortion minimization. 

#### 2.1 Least Squares Conformal Maps [(LSCM)](https://www.cs.jhu.edu/~misha/Fall09/Levy02.pdf)
As the name implies, LSCM is a **conformal** method, meaning it aims to minimize the conformal (angular) distortion of the parameterization, using a least squares solve. Deriving and computing this method by hand is beyond the scope of this exercise, so we will be using libigl's `lscm()` function to do the computation for us. 

One important note is that LSCM requires two vertices to be pinned in the plane to make the least-squared system well-determined (so there is a unique solution). Technically any two vertices can be chosen, but in practice vertices at the opposite end of a boundary loop are usually the best choice for method performance. The code commented below gives an example of computing the LSCM parameterization using libigl. 

In [ ]:
# TODO: Use the below example code to compute the LSCM parameterization of halfbunny.obj and ogre.obj
# from igl import boundary_loop, lscm

# bdry = boundary_loop(mesh.faces)

# b = np.array([bdry[0], bdry[int(len(bdry)/2)]], dtype="int")
# bc = np.array([[0, 0], [1, 1]], dtype=np.float32)
# succ, lscm_uv, error = lscm(mesh.vertices, mesh.faces, b, bc)

In [ ]:
# TODO: Compute the area, conformal, and isometric distortion for the LSCM parameterization of halfbunny.obj and ogre.obj

In [ ]:
# TODO: Compare the distortions of the LSCM results against the Tutte and Mean Value weights parameterizations
# The LSCM conformal result should be close to 0. What about the area and isometric distortions?

In [ ]:
# TODO: Visualize the distortion values using Polyscope

#### 2.2 As-Rigid-As-Possible Mesh Parameterization [(ARAP)](https://cs.harvard.edu/~sjg/papers/arap.pdf)
The ARAP method aims to minimize isometric distortion (both area and angles), using a non-linear algorithm which alternates between local and global optimization steps. The method requires an initial UV map as input, so we use a harmonic parameterization as an initial guess. 

In [ ]:
# TODO: Use the below example code to compute the ARAP parameterization of halfbunny.obj and ogre.obj
# from igl import ARAP, boundary_loop, harmonic, map_vertices_to_circle
# bnd = boundary_loop(mesh.faces)
# bnd_uv = map_vertices_to_circle(mesh.vertices, bnd).astype(mesh.vertices.dtype)
# initial_uv = harmonic(mesh.vertices, mesh.faces, bnd, bnd_uv, 1)
# arap = ARAP(mesh.vertices, mesh.faces, 2, np.zeros(0), with_dynamics=True)
# arap_uv = arap.solve(np.zeros((0,0)), initial_uv)

In [ ]:
# TODO: Compute the area, conformal, and isometric distortion for the LSCM parameterization of halfbunny.obj and ogre.obj

In [ ]:
# TODO: Compare the distortions of the ARAP results against the LSCM parameterization

In [ ]:
# TODO: Visualize the distortion values using Polyscope